Data Pipe

import packages

In [8]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.stats import norm

Define the inverse transform functuon

In [9]:
def inverse_transform_sampling(u):
    """Inverse transform sampling to convert uniformly distributed data into normally distributed data"""
    return norm.ppf(u)

Define the enhance features functuon

In [10]:
def enhance_features(df):
    """Add some features"""

    # 1. Net Goal Difference: Total goals scored last season - Total goals conceded last season
    df['Net_Goal_Difference'] = df['TotalGoalsLastSeason'] - df['TotalGoalsConcededLastSeason']

    # 2. Net Wins: Number of matches won last season - Number of matches lost last season
    df['Net_Wins'] = df['MatchesWonLastSeason'] - df['MatchesLostLastSeason']

    # 3. Effective Points Last Season: Matches won last season * 3 + Matches drawn last season * 1
    df['Effective_Points_Last_Season'] = df['MatchesWonLastSeason'] * 3 + df['MatchesDrawnLastSeason']

    # 4. Transfer Profit Last Season: Transfer income last season - Transfer spending last season
    df['Transfer_Profit_Last_Season'] = df['TransferIncomeLastSeason'] - df['TransferSpendingLastSeason']

    # 5. Revenue Ratio Last Season: Total revenue last season / Market value of the squad
    df['Revenue_Ratio_Last_Season'] = df['TotalRevenueLastSeason'] / df['MarketValueOfSquad']

    # 6. Team Efficiency: Total goals scored last season / Total goals conceded last season + Matches won last season / Matches lost last season
    # If the calculation cannot be performed, assign a value of 0
    df['Team_Efficiency'] = df.apply(lambda row: (
                row['TotalGoalsLastSeason'] / row['TotalGoalsConcededLastSeason'] + row['MatchesWonLastSeason'] / row[
            'MatchesLostLastSeason'])
    if (row['TotalGoalsConcededLastSeason'] != 0 and row['MatchesLostLastSeason'] != 0) else 0, axis=1)

    # 7. Estimated Attendance: Average attendance / Stadium capacity
    df['Estimated_Attendance'] =  df['AverageAttendance'] / df['StadiumCapacity']

    return df

Define the encode functuon

In [11]:
def encode_categorical_features(df, columns):
    """Encode categorical features with label encoding and one-hot encoding"""
    label_encoder = LabelEncoder()
    for column in columns:
        df[column + '_encoded'] = label_encoder.fit_transform(df[column])
        df[column] = df[column].astype(str)

    # Perform one-hot encoding
    encoded_df = pd.get_dummies(df, columns=columns)

    # Convert True/False in one-hot encoded results to 0/1
    for column in encoded_df.columns:
        if encoded_df[column].dtype == 'bool':
            encoded_df[column] = encoded_df[column].astype(int)

    return encoded_df

Define the normalization functuon

In [12]:
def normalize_data(df, column_name):
    """Normalize a specified column in a DataFrame"""
    normalized_data = pd.Series(index=df.index)
    transformations = {}

    for league, group in df.groupby('League_encoded'):
        data_column = group[column_name]

        # Check if the data is normally distributed
        ks_stat, ks_pvalue = stats.kstest(data_column.dropna().values, 'norm')

        if ks_pvalue > 0.05:
            # Use Z-Score normalization
            scaler = StandardScaler()
            normalized_group = scaler.fit_transform(data_column.dropna().values.reshape(-1, 1)).flatten()
            transformation = 'StandardScaler'
        else:
            # Use inverse transform sampling
            u = np.random.uniform(0, 1, size=len(data_column.dropna()))
            normalized_group = inverse_transform_sampling(u) * np.std(data_column.dropna()) + np.mean(data_column.dropna())

            
            # Use Z-Score normalization
            scaler = StandardScaler()
            normalized_group = scaler.fit_transform(normalized_group.reshape(-1, 1)).flatten()
            transformation = 'Z-score and Inverse Transform Sampling followed by StandardScaler'

        normalized_data.loc[group.index] = normalized_group
        transformations[league] = transformation

    return normalized_data, transformations

Define the main function

In [13]:
def main():
    # Read CSV file
    file_path = 'Football_teams_price_data.csv'
    df = pd.read_csv(file_path)

    # Add a data ID column and set it as the index
    df['dataID'] = df.index + 1
    df.set_index('dataID', inplace=True)

    # Add features
    df = enhance_features(df)

    # Encode categorical features
    columns_to_encode = ['League', 'Country', 'Manager', 'TeamFormation', 'PlayingStyle', 'HomeCity', 'StadiumType', 'MainSponsor', 'KitManufacturer', 'OwnershipType']
    encoded_df = encode_categorical_features(df, columns_to_encode)

    # Check and print the number of NaN values
    nan_count = encoded_df.isnull().sum()
    print("NaN values count per column:")
    print(nan_count)

    # Print the entire DataFrame
    print("\nDataFrame:")
    print(encoded_df)

    # Normalize data
    columns_to_normalize = [

        'AveragePlayerAge', 'TotalGoalsLastSeason', 'MatchesWonLastSeason',
        'MatchesDrawnLastSeason', 'MatchesLostLastSeason', 'TotalGoalsConcededLastSeason',
        'TotalRevenueLastSeason', 'StadiumCapacity', 'AverageAttendance',
        'TransferSpendingLastSeason', 'TransferIncomeLastSeason', 'NumberOfTrophies',
        'MarketValueOfSquad', 'AveragePlayerMarketValue', 'YouthAcademyRating', 'Net_Goal_Difference',
        'Net_Wins', 'Effective_Points_Last_Season', 'Transfer_Profit_Last_Season',
        'Revenue_Ratio_Last_Season', 'Team_Efficiency', 'Estimated_Attendance'

    ]
    transformations = {}
    for column in columns_to_normalize:
        normalized_column, column_transformations = normalize_data(encoded_df, column)
        encoded_df[column] = normalized_column
        transformations[column] = column_transformations

    # Print the transformation method for each column
    for column, column_transformations in transformations.items():
        print(f"Column: {column}, Transformations: {column_transformations}")

    # Min-Max normalization for the Price column
    encoded_df['Price'] = pd.to_numeric(encoded_df['Price'].str.replace(',', ''), errors='coerce')
    scaler = MinMaxScaler()
    encoded_df['Price'] = scaler.fit_transform(encoded_df[['Price']])

    # Subtract the minimum value from all columns
    for column in encoded_df.columns:
        if encoded_df[column].dtype in ['int64', 'float64']:  # Ensure it's a numeric column
            encoded_df[column] -= encoded_df[column].min()

    # Save the result to a CSV file
    encoded_df.to_csv('result.csv')

Run the code

In [14]:
if __name__ == "__main__":
    main()

NaN values count per column:
AveragePlayerAge                  0
TotalGoalsLastSeason              0
MatchesWonLastSeason              0
MatchesDrawnLastSeason            0
MatchesLostLastSeason             0
                                 ..
KitManufacturer_Manufacturer C    0
OwnershipType_Consortium          0
OwnershipType_Fan-Owned           0
OwnershipType_Private             0
OwnershipType_Public              0
Length: 76, dtype: int64

DataFrame:
        AveragePlayerAge  TotalGoalsLastSeason  MatchesWonLastSeason  \
dataID                                                                 
1              24.367182                    28                    16   
2              34.162143                    89                    12   
3              30.443897                    43                     3   
4              28.177194                    84                    19   
5              20.652317                    36                    36   
...                  ...          